# What is the best way to create residuals in a convolution network? 
This notebook shows two different ways to create residual shortcuts in your network that can presumably improve the effectiveness of backpropagation through the network. 


### Questions

1. What are the advantages of each method?
2. What is the best practice for reshaping 3D tensors to a standard size?

In [30]:
import keras
from keras.layers import Input, LSTM, Dense, merge
from keras.models import Model
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, SimpleRNN, Reshape, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense


## 1. Use convolution layers as residual shortcut.

This example shows how 3D tensors from convolution layers can be used as residual shortcuts. The difficult part of this method is keeping the shapes of the tensors the same size as the convolved tensors become narrower and deeper. There are several examples showing how an extra 1x1 convolution  layer can be used to match the size of the residual tensors. 
* [Keras Resnet](https://github.com/raghakot/keras-resnet/blob/master/resnet.py#L41)



In [114]:
img_in = Input(shape=(120, 160, 3), name='img_in')

net =  Convolution2D(64, 6, 6, subsample=(3,3), name='conv1')(img_in)
net =  Dropout(.2)(net)
aux1 = Dense(64, name='aux1')(net)

net =  Convolution2D(64, 3, 3, subsample=(1,1), border_mode='same', name='conv2')(net)
net =  Dropout(.2)(net)

net = Dense(64)(net)

net = merge([net, aux1], mode='sum')
net = Flatten()(net)
angle_out = Dense(1, name='angle_out')(net)
model = Model(input=[img_in], output=[angle_out])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
img_in (InputLayer)              (None, 120, 160, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 39, 52, 64)    6976        img_in[0][0]                     
____________________________________________________________________________________________________
dropout_222 (Dropout)            (None, 39, 52, 64)    0           conv1[0][0]                      
____________________________________________________________________________________________________
conv2 (Convolution2D)            (None, 39, 52, 64)    36928       dropout_222[0][0]                
___________________________________________________________________________________________

## 2. Using flat+dense layers as residual shortcut
Another way to create a residual shortcut is to flatten the tensor and use a dense layer to standardize it's size. This method has an order of magnitude more parameters to train but is easy to implement. This method was used in the [vision module](https://github.com/udacity/self-driving-car/blob/master/steering-models/community-models/komanda/solution-komanda.ipynb) of winner of the last Udacity Self Driving Car steering angle challenge.

In [117]:
img_in = Input(shape=(120, 160, 3), name='img_in')

net =  Convolution2D(64, 6, 6, subsample=(3,3), name='conv0')(img_in)
net =  Dropout(.2)(net)

#Create residual to shortcut
aux1 = Flatten(name='aux1_flat')(net)
aux1 = Dense(64, name='aux1_dense')(aux1) 

net =  Convolution2D(128, 3, 3, subsample=(2,2), border_mode='same', name='conv2')(net)
net =  Dropout(.2)(net)
net = Flatten()(net)
net = Dense(64)(net)

net = merge([net, aux1], mode='sum')

angle_out = Dense(1, name='angle_out')(net)
model = Model(input=[img_in], output=[angle_out])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
img_in (InputLayer)              (None, 120, 160, 3)   0                                            
____________________________________________________________________________________________________
conv0 (Convolution2D)            (None, 39, 52, 64)    6976        img_in[0][0]                     
____________________________________________________________________________________________________
dropout_228 (Dropout)            (None, 39, 52, 64)    0           conv0[0][0]                      
____________________________________________________________________________________________________
conv2 (Convolution2D)            (None, 20, 26, 128)   73856       dropout_228[0][0]                
___________________________________________________________________________________________